<a href="https://colab.research.google.com/github/RashikaTyagi01/Rashika-UML501/blob/main/mlAssignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q1. Write a Python program to scrape all available books from the website
(https://books.toscrape.com/) Books to Scrape – a live site built for practicing scraping (safe,
legal, no anti-bot). For each book, extract the following details:
1. Title
2. Price
3. Availability (In stock / Out of stock)
4. Star Rating (One, Two, Three, Four, Five)
Store the scraped results into a Pandas DataFrame and export them to a CSV file named
books.csv.
(Note: Use the requests library to fetch the HTML page. Use BeautifulSoup to parse and extract
book details and handle pagination so that books from all pages are scraped)

In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [24]:
base_url = "https://books.toscrape.com/catalogue/page-{}.html"
books = []
page = 1
while True:
   url = base_url.format(page)
   response = requests.get(url)
   if response.status_code != 200:
        break
   soup = BeautifulSoup(response.text, "html.parser")
   book_items = soup.find_all("article", class_="product_pod")
   if not book_items:
        break
   for item in book_items:
    title = item.h3.a["title"]
    price = item.find("p", class_="price_color").text.strip()
    availability = item.find("p", class_="instock availability").text.strip()
    rating=item.p["class"][1]
    books.append([title, price, availability, rating])

   page+=1
df=pd.DataFrame(books, columns=["Title", "Price", "Availability", "Star Rating"])
df.to_csv("books.csv", index=False)
print(df.head())



                                   Title    Price Availability Star Rating
0                   A Light in the Attic  Â£51.77     In stock       Three
1                     Tipping the Velvet  Â£53.74     In stock         One
2                             Soumission  Â£50.10     In stock         One
3                          Sharp Objects  Â£47.82     In stock        Four
4  Sapiens: A Brief History of Humankind  Â£54.23     In stock        Five


Q2. Write a Python program to scrape the IMDB Top 250 Movies list
(https://www.imdb.com/chart/top/) . For each movie, extract the following details:
1. Rank (1–250)
2. Movie Title
3. Year of Release
4. IMDB Rating
Store the results in a Pandas DataFrame and export it to a CSV file named imdb_top250.csv.
(Note: Use Selenium/Playwright to scrape the required details from this website)

In [ ]:
!pip install selenium


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.15.0
    Uninstalling typing_extensions-4.15.0:
      Successfully uninstalled typing_extensions-4.15.0


In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

chrome_opts = Options()
chrome_opts.add_argument("--headless")
chrome_opts.add_argument("--no-sandbox")
chrome_opts.add_argument("--disable-dev-shm-usage")
chrome_opts.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/115.0.0.0 Safari/537.36"
)
browser = webdriver.Chrome(options=chrome_opts)
browser.get("https://www.imdb.com/chart/top/")
time.sleep(5)
film_list = []
movie_cards = browser.find_elements(By.CSS_SELECTOR, ".ipc-metadata-list-summary-item")
for rank, card in enumerate(movie_cards, start=1):
    try:
        name = card.find_element(By.CSS_SELECTOR, "h3").text
        release_year = card.find_element(By.CSS_SELECTOR, ".cli-title-metadata-item").text
        score = card.find_element(By.CSS_SELECTOR, ".ipc-rating-star--imdb").text.split()[0]
        film_list.append([rank, name, release_year, score])
    except Exception as e:
        print(f"Skipping a card due to error: {e}")
browser.quit()
imdb_table = pd.DataFrame(film_list, columns=["Position", "Movie Title", "Release Year", "Rating"])
imdb_table.to_csv("imdb_top250.csv", index=False)
print(imdb_table.head())

   Position                  Movie Title Release Year Rating
0         1  1. The Shawshank Redemption         1994    9.3
1         2             2. The Godfather         1972    9.2
2         3           3. The Dark Knight         2008    9.1
3         4     4. The Godfather Part II         1974    9.0
4         5              5. 12 Angry Men         1957    9.0


Q3. Write a Python program to scrape the weather information for top world cities from the
given website (https://www.timeanddate.com/weather/) . For each city, extract the following
details:
1. City Name
2. Temperature
3. Weather Condition (e.g., Clear, Cloudy, Rainy, etc.)
Store the results in a Pandas DataFrame and export it to a CSV file named weather.csv.

In [30]:

url = "https://www.timeanddate.com/weather/"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
table = soup.find("table")
if table is None:
    print("No table found")
data = []
for row in table.find_all("tr"):
    cols = row.find_all("td")
    if len(cols) >= 4:
        city = cols[0].get_text(strip=True)
        temperature = cols[2].get_text(strip=True)
        condition = cols[3].get_text(strip=True)
        data.append({"City": city, "Temperature": temperature, "Condition": condition})

df = pd.DataFrame(data)
df.to_csv("weather.csv", index=False)
print(df.head())


          City Temperature Condition
0        Accra                 79 °F
1  Addis Ababa                 64 °F
2     Adelaide                 55 °F
3      Algiers                 78 °F
4       Almaty                 55 °F
